In [2]:

from common.read_spectral_common import (
    data_df_13_09_2022, 
    calculate_mutual_info_for_all, 
    generate_sample, create_X_train_Y_train, 
    mutual_info_regression, 
    get_max_bands, 
    get_bands_ix_from_mutual_info, 
    get_average_bands, 
    get_max_bands, 
    get_min_bands,
    predict_using_neutral_network, 
    predict_using_random_forest, 
    predict_using_decision_tree,
    get_full_path,
    load_sklearn_model_to_file_by_cloudpickle,
)
import matplotlib.pyplot as plt
import pickle
import cloudpickle
import common
from sklearn import tree
from sklearn.metrics import mean_squared_error
import numpy as np
from datetime import datetime
import os
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd

In [14]:

target_value = "P"
train_field = "T"
function_get = get_max_bands

In [15]:
calculate_mutual_info_for_all(data_df_13_09_2022, target_value, train_field, function_get)

[(29, 0.40083381562669373),
 (27, 0.23341104691909154),
 (9, 0.21625734506316263),
 (92, 0.1923746727515736),
 (14, 0.16710453698817895),
 (25, 0.16582254665974006),
 (107, 0.14230151466555396),
 (87, 0.14155293403733005),
 (59, 0.13599192896192047),
 (91, 0.13298349673340537),
 (93, 0.131278457677666),
 (111, 0.131174691384496),
 (39, 0.09026556349259618),
 (36, 0.09024680046851863),
 (68, 0.08545210070713471),
 (112, 0.07868734542378597),
 (115, 0.07642502842525545),
 (86, 0.07632789644910343),
 (69, 0.07624256907503657),
 (53, 0.07530437930269285),
 (16, 0.07263162461068307),
 (38, 0.06966971297006053),
 (10, 0.0671481306624293),
 (110, 0.06663090085402024),
 (76, 0.06547028390629128),
 (4, 0.0623468273708605),
 (8, 0.06120796084188296),
 (79, 0.06073088737201138),
 (32, 0.06031793731686674),
 (81, 0.056432020139036254),
 (58, 0.052839689351315666),
 (82, 0.05274346937187868),
 (47, 0.04888824889377563),
 (35, 0.048328357012977996),
 (88, 0.048193546511275276),
 (56, 0.0442497402137

In [16]:
bands_ix = get_bands_ix_from_mutual_info(data_df_13_09_2022, -1, target_value, train_field, function_get)

In [6]:
len(bands_ix)

122

In [17]:
# bands_ix = filter(lambda data: data < 100, bands_ix)

In [7]:
bands_ix = list(bands_ix)

In [8]:
bands_ix

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121]

In [17]:
len(bands_ix)

122

In [10]:
date_now = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
NN_file_path = get_full_path(f"../../model_saved/NN_save/NN_object/{'_'.join(train_field)}_predict_{'_'.join(target_value)}_{date_now}_using_{function_get.__name__}.pkl")
RF_file_path = get_full_path(f"../../model_saved/RF_save/{'_'.join(train_field)}_predict_{'_'.join(target_value)}_{date_now}_using_{function_get.__name__}.pkl")
DT_file_path = get_full_path(f"../../model_saved/DT_save/{'_'.join(train_field)}_predict_{'_'.join(target_value)}_{date_now}_using_{function_get.__name__}.pkl")

In [22]:
sample = generate_sample(data_df_13_09_2022, bands_ix, target_value, train_field)
X_train, Y_train = create_X_train_Y_train(sample, bands_ix)
sample_target = generate_sample(data_df_13_09_2022, bands_ix, target_value, "BC")
X_target, Y_target = create_X_train_Y_train(sample_target, bands_ix)
super_param={"lr": 0.0001, "weight_decay": 1e-5, "n_epochs": 40000, "stop_value": 130}
re_run = "Y"
loss_NN, pred_NN, NN_model = predict_using_neutral_network(
    X_train, 
    Y_train, 
    X_target, 
    Y_target, 
    bands_ix,
    NN_file_path, 
    super_param, 
    re_run)
print(f"{loss_NN=}")
print(f"{pred_NN=}")
loss_RF, pred_RF, RF_model = predict_using_random_forest(X_train, Y_train, X_target, Y_target, bands_ix, super_param)
print(f"{loss_RF=}")
print(f"{pred_RF=}")
loss_DT, pred_DT, DT_model = predict_using_decision_tree(X_train, Y_train, X_target, Y_target, bands_ix, super_param)
print(f"{loss_DT=}")
print(f"{pred_DT=}")

Epoch: 0 | loss train: 14027.6005859375
Epoch: 100 | loss train: 13697.357421875
Epoch: 200 | loss train: 2434.593505859375
Epoch: 300 | loss train: 2424.35498046875
Epoch: 400 | loss train: 2418.376953125
Epoch: 500 | loss train: 2407.356689453125
Epoch: 600 | loss train: 2392.379638671875
Epoch: 700 | loss train: 2375.621826171875
Epoch: 800 | loss train: 2357.351806640625
Epoch: 900 | loss train: 2336.76904296875
Epoch: 1000 | loss train: 2311.5068359375
Epoch: 1100 | loss train: 2261.446044921875
Epoch: 1200 | loss train: 2189.191162109375
Epoch: 1300 | loss train: 2158.31103515625
Epoch: 1400 | loss train: 2141.019775390625
Epoch: 1500 | loss train: 2115.516357421875
Epoch: 1600 | loss train: 2074.738525390625
Epoch: 1700 | loss train: 2018.156982421875
Epoch: 1800 | loss train: 1941.8135986328125
Epoch: 1900 | loss train: 1865.195068359375
Epoch: 2000 | loss train: 1801.3253173828125
Epoch: 2100 | loss train: 1753.5562744140625
Epoch: 2200 | loss train: 1713.948486328125
Epoch: 2

In [23]:
cloudpickle.register_pickle_by_value(common)

with open(NN_file_path, "wb") as file:
    cloudpickle.dump(NN_model, file)
    print(f"Save NN object to: {NN_file_path}")
with open(RF_file_path, "wb") as file:
    cloudpickle.dump(RF_model, file)
    print(f"Save RF object to: {RF_file_path}")
with open(DT_file_path, "wb") as file:
    cloudpickle.dump(DT_model, file)
    print(f"Save DT object to: {DT_file_path}")

Save NN object to: D:\code\model_saved\NN_save\NN_object\T_predict_K_2024-03-07-22-56-00_using_get_max_bands.pkl
Save RF object to: D:\code\model_saved\RF_save\T_predict_K_2024-03-07-22-56-00_using_get_max_bands.pkl
Save DT object to: D:\code\model_saved\DT_save\T_predict_K_2024-03-07-22-56-00_using_get_max_bands.pkl


In [24]:
avarage_target_value = np.average(Y_target)
loss_NN = float(loss_NN)
df_report = pd.DataFrame(
    {
        "date_time": [datetime.now()],
        "train_field": [str(train_field)],
        "target_value": [str(target_value)],
        "bands_ix": [str(bands_ix)],
        "total_band": [len(bands_ix)],
        "function_get": [function_get.__name__],
        "loss_NN": [loss_NN],
        "% loss_NN": [loss_NN / avarage_target_value * 100],
        "loss_RF": [loss_RF],
        "% loss_RF": [loss_RF / avarage_target_value * 100],
        "loss_DT": [loss_DT],
        "% loss_DT": [loss_DT / avarage_target_value * 100],
        "average_target_value": [avarage_target_value],
        "File save NN": [NN_file_path],
        "File save RF": [RF_file_path],
        "File save DT": [DT_file_path],
    }
)

In [25]:
df_report

,date_time,train_field,target_value,bands_ix,total_band,function_get,loss_NN,% loss_NN,loss_RF,% loss_RF,loss_DT,% loss_DT,average_target_value,File save NN,File save RF,File save DT
0,2024-03-07 23:02:15.481179,T,K,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",122,get_max_bands,4755.72998,37.707614,3506.90181,27.805805,3099.819108,24.578095,12612.121094,D:\code\model_saved\NN_save\NN_object\T_predic...,D:\code\model_saved\RF_save\T_predict_K_2024-0...,D:\code\model_saved\DT_save\T_predict_K_2024-0...


In [26]:
def append_df_to_excel(df: pd.DataFrame, file_path, sheet_name="Sheet1"):
    if not os.path.exists(file_path):
        df.to_excel(
            file_path,
            sheet_name=sheet_name,
            index=False,
            header=True
        )
        return
    
    wb = load_workbook(file_path)
    ws = wb[sheet_name]
    for r in dataframe_to_rows(df, index=False, header=False):
        ws.append(r)
    wb.save(file_path)

In [27]:
try:
    report_file_path = get_full_path("../../report/agriculture_report.xlsx")
    sheet_name = "Sheet1"

    append_df_to_excel(df_report, report_file_path, sheet_name)
    print(f"SUCCESS: DATA IS WRITED TO FILE {report_file_path}")
except PermissionError as err:
    print(f"ERROR: YOU ARE OPENNING FILE {report_file_path}")

SUCCESS: DATA IS WRITED TO FILE D:\code\report\agriculture_report.xlsx
